In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [ ]:
### zbp total with features data
file_path = '../../../src/data/temp/lagged_zbp_totals_with_features.csv'
data = pd.read_csv(file_path)

## unemployment data
file_path = '../../../src/data/temp/processed_unemployment_data.csv'
unemployment_data = pd.read_csv(file_path)

## Correlation Matrix

In [ ]:
numeric_columns = data.select_dtypes(include=['number']).columns
selected_df = data[numeric_columns]

correlation_matrix = selected_df.corr()

In [ ]:
correlation_matrix.head()

In [ ]:
correlation_with_est = correlation_matrix['est'].sort_values(ascending=False)

top_5_features = correlation_with_est.head(6)[1:]  # Excluding 'est' 

print(top_5_features)

## Using the Features Used in rf_reg_model

In [ ]:
end_year = 2020
data_train = data[data['year'] <= end_year]
data_test = data[data['year'] > end_year]
included_feats = ['zip', 'year', 'naics_11_pct', 'naics_21_pct', 'naics_22_pct', 'naics_23_pct',
                   'naics_31_pct', 'naics_42_pct', 'naics_44_pct', 'naics_48_pct',
                   'naics_51_pct', 'naics_52_pct', 'naics_53_pct', 'naics_54_pct',
                   'naics_55_pct', 'naics_56_pct', 'naics_61_pct', 'naics_62_pct',
                   'naics_71_pct', 'naics_72_pct', 'naics_81_pct', 'naics_99_pct',
                   'n1_4_pct', 'n5_9_pct', 'n10_19_pct', 'n20_49_pct', 'n50_99_pct',
                   'n100_249_pct', 'n250_499_pct', 'n500_999_pct', 'n1000_pct']
X_train = data_train[included_feats]
y_train = data_train['est']
X_test = data_test[included_feats]
y_test = data_test['est']

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

## Using the Top 5 Features From Correlation Matrix

In [ ]:
included_feats = ['emp', 'ap', 'qp1', 'naics_51_pct', 'n10_19_pct']
X_train = data_train[included_feats]
y_train = data_train['est']
X_test = data_test[included_feats]
y_test = data_test['est']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

## Using the Top 10 Features

In [ ]:
correlation_with_est = correlation_matrix['est'].sort_values(ascending=False)

top_10_features = correlation_with_est.head(11)[1:]

print(top_10_features)

In [ ]:
included_feats = ['emp', 'ap', 'qp1', 'naics_51_pct', 'n10_19_pct', 'n5_9_pct', 'naics_31_pct', 'n20_49_pct', 'naics_52_pct','naics_62_pct']
X_train = data_train[included_feats]
y_train = data_train['est']
X_test = data_test[included_feats]
y_test = data_test['est']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Fixed Effect Model

In [ ]:
# Create dummy variables for each ZIP code
dummies = pd.get_dummies(data['zip'], drop_first=True).astype(int)

# Concatenate the dummy variables with the original data
data_panel = pd.concat([data.drop(columns=['zip']), dummies], axis=1)

X = data_panel.drop(columns=['est', 'emp_nf', 'qp1_nf', 'ap_nf']) 
y = data_panel['est']  # Dependent variable predicting establishment growth

# Constant term to the independent variables
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

## Statistically Significant Values

- em with a p-value of 0.0000
- ap with a p-value of 0.006
- n1_4_pct with a p-value < 0.000
- n5_9_pct with a p-value < 0.000
- n10_19_pct with a p-value < 0.000
- n20_49_pct with a p-value < 0.000
- n50_99_pct with a p-value < 0.000

## Interesting Zip Code

### - 91910 p-value (0.744)
### - 91911 p-value (0.385)

This output indicates that the model has a high R-squared value **(0.998)**, suggesting that the independent variables explain a large portion of the variance in the dependent variable.

# Random Effect Model

In [ ]:
X = data_panel.drop(columns=['est', 'emp_nf', 'qp1_nf', 'ap_nf']) 
y = data_panel['est']  

X = sm.add_constant(X)

model = sm.regression.mixed_linear_model.MixedLM(y, X, groups=data['zip'])

# Specify the optimizer (e.g., 'nm' for Nelder-Mead)
optimizer = 'nm'

iterations = 1000

mixed_model_fit = model.fit(method=optimizer, maxiter=iterations)

print(mixed_model_fit.summary())

## Statistically Significant Values

- ap with p-value 0.006
- n1_4_pct with p-value 0.000
- n5_9_pct with p-value 0.000
- n10_19_pct with p-value 0.000
- n20_49_pct with p-value 0.000
- All zipcodes

## Summary 

**Significance of the individual-specific effects**: The p-values for the individual-specific effects in the fixed effects model are statistically significant, indicating that there is likely unobserved heterogeneity at the individual level that affects the outcome variable. Suggesting that there are **individual-specific** characteristics or factors that are important to consider and control for in your analysis.

**Adjusted R-squared**: The adjusted R-squared value for the fixed effects model is higher compared to the random effects model. This suggests that the fixed effects model explains a greater proportion of the variation in the outcome variable.

Overall **fixed effect model** seems better for the data